# IA369 Y - T2

Filipe Antonio de Barros Reis - RA 091202
<br>Lucas Brugnaro Badur - RA 094906

# Sobre
Projeto realizado para a disciplina de pós-graduação __IA369Y - Computação Afetiva__ oferecida pela Faculdade de Engenharia Elétrica (FEEC) da Universidade Estadual de Campinas (UNICAMP), lecionada pela professora Paula Dornhofer Paro Costa. 

### Aviso

Este projeto tem como objetivo explorar possibilidades de processamento de linguagem natural para o português brasileiro. Todas as abordagens visam maximizar o aprendizado e a compreensão dos métodos utilizados, e não a performance.


# Introdução

Duas propostas foram disponibilizadas para esse projeto, a de classificar valência de manchetes brasileiras disponibilizadas em um conjunto de dados e a de atribuir intensidade de diferentes sentimentos a manchetes americanas.
<br>A primeira opção foi a escolhida pelos autores devido à sua complexidade relacionada à diminuta disponibilidade de corpus e referências para o processamento de linguagem natural no idioma Português Brasileiro.
<br>Com isso, este projeto tem como objetivo servir como uma referência básica para o processamento de linguagem natural em português, utilizando como uma alternativa viável a tradução para o inglês, já que para manchetes curtas e sentenças simples a tradução é mais do que adequada, sem comprometer o sentimento geral na maioria dos casos.
    
Além das dificuldades impostas pelo idioma, existe uma dificuldade inerente à tarefa de analisar valência em manchetes, já que elas são idealmente imparciais, dado o comprometimento ético e com neutralidade que a mídia jornalística deveria apresentar. Por outro lado, ao julgar a valência da manchete, o leitor tipicamente associa seu conteúdo às suas vivências anteriores e visão de mundo.  
    
Esse projeto apresenta diferentes abordagens para o processamento de linguagem natural em português, bem como os pontos fortes e fracos de cada uma das abordagens utilizadas neste projeto. Por fim, também são apresentadas abordagens para um classificador de origem do jornal e análises de qual das publicações obteve resultados mais neutros.

# Abordagens adotadas
Nesse projeto, três abordagens foram consideradas:
- Tradução do banco de manchetes para inglês e uso da biblioteca TextBlob: essa estratégia foi considerada dada sua simplicidade e facilidade de implementação, sendo ótima para um trabalho inicial;
- Avaliação de sentenças baseada em unigramas utilizando tanto o léxico em português Sentilex, quanto traduzindo e considerando o SentiWordNet;
- Avaliação de sentenças inteiras em inglês, utilizando o módulo Vader.

In [1]:
%config IPCompleter.greedy=True #Turning the autocomplete on
import nltk #used for text processing
import pandas as pd #used for csv and matrix manipúlation
import matplotlib as plt #used for data visualization
import numpy as np #used for numerical application
import os #used for folder checks
import sys #used for folder checks
from googletrans import Translator #translation lib
from textblob import TextBlob #data processing lib
from textblob.sentiments import NaiveBayesAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer #sentence sentiment analysis library
import re #used for regular expressions

Vamos começar lendo o arquivo csv processado utilizando regexp no editor SublimeText

In [2]:
originalDb = pd.read_csv("manchetesBrasildatabase/manchetesBrasildatabase.csv", 
                         names=["Day", "Month", "Year", "Source", "Headline"])

# Considerações Gerais

## Separação em sentenças

Uma abordagem clássica da análise de textos é utilizar mecanismos para separá-los em sentenças e trabalhar com essas de forma independente. Como as manchetes são usualmente compostas por apenas uma sentença, não faremos nenhum tratamento para separação do texto em sentenças.
<br>Na seção que utiliza a biblioteca TextBlob para análise, uma avaliação prévia nos demonstra que no caso de realizar uma separação em sentenças, apenas 7 manchetes seriam quebradas em mais de uma sentença.

# Translator + TextBlob

As duas abordagens de classificação diferem em sua essência, já que a por _Naive Bayes_ é baseada em um classificador treinado a partir de avaliações de filmes, contrastando com a utilizando a biblioteca _Pattern_ que é baseada em contagem de palavras utilizadas e sentimento associado a elas, com treinamento realizado a partir de avaliações de produtos.

A documentação completa do TextBlob está disponível em: http://textblob.readthedocs.io/en/dev/advanced_usage.html
<br>A documentação para o método _Pattern_ do TextBlob pode ser lida em: https://www.clips.uantwerpen.be/pages/pattern-en#sentiment

In [3]:
translator = Translator()
translated = []
if os.path.isfile('translated.csv'):
    translatedDb = pd.read_csv('translated.csv', encoding='latin-1')
else:
    for element in originalDb.Headline:
        currentTranslation = translator.translate(element) 
        sys.stdout.write("\r" + currentTranslation.text)
        sys.stdout.flush()
        translated.append(currentTranslation.text)
        
    translatedDb = originalDb
    translatedDb["THeadline"] = translated
    translatedDb.to_csv('translated.csv')
    
display (translatedDb.head(5))

,Unnamed: 0,Day,Month,Year,Source,Headline
0,0,1,'fevereiro',2017,'Valor','BNDES shrinks back to the level of 20 years ago'
1,1,1,'fevereiro',2017,'Valor','BC creates new monetary policy instrument.'
2,2,1,'fevereiro',2017,'Valor','Exchange generates word of mouth between AU a...
3,3,1,'fevereiro',2017,'Valor','Indemnification to energy transmitters has al...
4,4,1,'fevereiro',2017,'Valor',"""Politicians expect rapporteur to separate"" wh..."


Para avaliar se as manchetes devem ser separadas por sentenças, vamos contar o total de sentenças e comparar com o total de manchetes.

In [4]:
sentencesCount = 0
for headline in translatedDb.Headline:
        analysis = TextBlob(headline )
        sentencesCount += len(analysis.sentences)
print("Sentenças: {0} \nManchetes: {1}".format(sentencesCount, len(translatedDb)))

Sentenças: 507 
Manchetes: 500


Como o número de sentenças excede o de manchetes em apenas 7 unidades, é justificável não realizarmos essa separação dada a complexidade adicionada ao processo e o reduzido ganho.

Agora podemos partir para a classificação das manchetes. Como já mencionado, essa biblioteca disponibiliza duas opções de análise de sentimentos:
- Pattern: biblioteca que implementa a análise de sentimento utilizando um lexicon composto por palavras recorrentes em análises de produtos, anotadas de acordo com sua polaridade e subjetividade;
- Naive Bayes: implementação do algoritmo de _Naive Bayes_ implementado pela biblioteca __nltk__ e treinado com uma base de dados de avaliações de filmes. Um ponto bastante negativo dessa implementação (ou ao menos do formato que utilizamos) é que o classificador é treinado novamente para cada frase considerada, exigindo um longo tempo de execução. 

A fim de evitar processamentos adicionais descecessários, implementamos o bloco abaixo que verifica se as machetes já foram processadas em algum momento e caso já tenham sido, apenas carrega os resultados para agilizar a execução.

In [5]:
if os.path.isfile('translated_bayes.csv'):
    translatedDb = pd.read_csv('translated_bayes.csv', encoding='latin-1', index_col=0)
else:
    sentiments = []
    for headline in translatedDb.Headline:
        analysis = TextBlob(headline )
        sys.stdout.write("\r" + headline + str(analysis.sentiment))
        sys.stdout.flush()
        sentiments.append(analysis.sentiment)
    translatedDb['BlobSentimentsP']= sentiments
    print('\n\n\n*******Finished Pattern method, Starting Naive Bayes\n This may take a while.')
    sentiments = []
    for headline in translatedDb.Headline:
        analysis = TextBlob(headline, analyzer=NaiveBayesAnalyzer() )
        sys.stdout.write("\r" + headline + str(analysis.sentiment))
        sys.stdout.flush()
        sentiments.append(analysis.sentiment)
    translatedDb['BlobSentimentsNB']= sentiments
    print (len(translatedDb))
    translatedDb.head()
    translatedDb.to_csv('translated_bayes.csv')

Para facilitar a visualização de alguns resultados, podemos ordenar por ordem de magnitude dos sentimentos calculados utilizando a biblioteca pattern:

In [6]:
trnsSorted = translatedDb.sort_values(['BlobSentimentsP'])
print("Lowest: \n")
display(trnsSorted.head(5))
print("\n\n\nHighest: \n")
display(trnsSorted.tail(5))

Lowest: 



,Unnamed: 0.1,Day,Month,Year,Source,Headline,BlobSentimentsP,BlobSentimentsNB
248,248,14,'junho',2017,'Folha','STF must judge Aecio's arrest warrant on the ...,"Sentiment(polarity=-0.025, subjectivity=0.0)","Sentiment(classification='neg', p_pos=0.237896..."
374,374,17,'janeiro',2017,'Globo','Prison under arrest has been imprisoned outsi...,"Sentiment(polarity=-0.025, subjectivity=0.025)","Sentiment(classification='pos', p_pos=0.853506..."
329,329,15,'março',2017,'Folha','Former president testifies and says he is the...,"Sentiment(polarity=-0.037500000000000006, subj...","Sentiment(classification='neg', p_pos=0.433073..."
203,203,3,'julho',2017,'Valor','Government studies other measures for social ...,"Sentiment(polarity=-0.04583333333333334, subje...","Sentiment(classification='pos', p_pos=0.952242..."
64,64,1,'junho',2017,'Globo','Marcus Aurelius will take Aetius' arrest requ...,"Sentiment(polarity=-0.05, subjectivity=0.0)","Sentiment(classification='pos', p_pos=0.940510..."





Highest: 



,Unnamed: 0.1,Day,Month,Year,Source,Headline,BlobSentimentsP,BlobSentimentsNB
7,7,1,'fevereiro',2017,'Globo','Fachin will be able to go to a class that thi...,"Sentiment(polarity=0.5, subjectivity=0.625)","Sentiment(classification='pos', p_pos=0.746054..."
390,390,17,'janeiro',2017,'Estado','Executives are confident but do not intend to...,"Sentiment(polarity=0.5, subjectivity=0.8333333...","Sentiment(classification='pos', p_pos=0.732663..."
51,51,1,'março',2017,'Estado','Tatuapé celebrates an unprecedented title.',"Sentiment(polarity=0.6, subjectivity=0.9)","Sentiment(classification='pos', p_pos=0.564443..."
456,456,23,'agosto',2017,'Estado','The 30 Best Series of All Time According to R...,"Sentiment(polarity=1.0, subjectivity=0.3)","Sentiment(classification='pos', p_pos=0.589663..."
458,458,23,'agosto',2017,'Estado','BBC chooses the 100 best comedies of all time.',"Sentiment(polarity=1.0, subjectivity=0.3)","Sentiment(classification='pos', p_pos=0.664478..."


É possível percebermos que o resultado muda do método Pattern para o Naive Bayes. Um exemplo claro dessa variação é encontrado na linha 374, onde a manchete "Prisão rebelada tem presos fora das celas desde 2015." foi classificada como negativa pelo Pattern e positiva pelo NaiveBayes. Uma possível justificativa para isso é que palavras como "rebelada", "prisão" e "celas" podem estar relacionadas a filmes bons, mas não possuem relação com bons produtos.
A análise detalhada e comparação aprofundada dos resultados pode ser encontrada na seção Resultados.

# Bag of Words + SentiLex

Nessa abordagem utilizamos o SentiLex-PT01, que é um léxico de sentimentos para o português de portugal criado pela Universidade de Lisboa e formado por 6.321 palavras. Para a utilização de léxico, adotamos a abordagem de reduzir as manchetes a grupos de palavras individuais, remover as _stopwords_, classificar essas palavras de forma individual e avaliar a valência da frase tomando por base o conjunto de pontuações individuais. 
<br>Para isso, iniciamos carregando o léxico:

In [7]:
sentiLex = pd.read_csv('SentiLex-PT01/SentiLex-lem-PT01.txt', sep = ';', names=['Word', 'GN', "TG", "Pol", "Anot"])
display(sentiLex.head(5))

,Word,GN,TG,Pol,Anot
0,abafado.PoS=Adj,GN=ms,TG=HUM,POL=-1,ANOT=JALC
1,abafante.PoS=Adj,GN=ms,TG=HUM,POL=-1,ANOT=MAN
2,abaixado.PoS=Adj,GN=ms,TG=HUM,POL=-1,ANOT=JALC
3,abalado.PoS=Adj,GN=ms,TG=HUM,POL=-1,ANOT=JALC
4,abalizado.PoS=Adj,GN=ms,TG=HUM,POL=1,ANOT=JALC


Como o arquivo carregado está com as informações e formatação padrão do léxico, precisamos adequá-lo ao nosso uso. Para tanto, podemos remover a função gramatical associada às palavras e o indicador POL= do campo de polaridade das mesmas.

In [8]:
newWords = []
newPols= []
for line in sentiLex.Word:
    newWords.append(line.replace('.PoS=Adj',''))
for line in sentiLex.Pol:
    newPols.append(line.replace('POL=',''))

sentiLex.Word = newWords
sentiLex.Pol = newPols
display(sentiLex.head(5))

,Word,GN,TG,Pol,Anot
0,abafado,GN=ms,TG=HUM,-1,ANOT=JALC
1,abafante,GN=ms,TG=HUM,-1,ANOT=MAN
2,abaixado,GN=ms,TG=HUM,-1,ANOT=JALC
3,abalado,GN=ms,TG=HUM,-1,ANOT=JALC
4,abalizado,GN=ms,TG=HUM,1,ANOT=JALC


Agora temos uma base no formato desejado. 

O próximo passo é preparar nossas manchetes para a análise. Para tanto, podemos remover alguns caracteres não-importantes, como numerais. Os demais caracteres especiais também podem ser removidos, mas o tratamento para remover apenas caracteres especiais excluindo pontuações do escopo do projeto.

In [9]:
bowDb = originalDb
wordLines = []
def rem_numbers(text):
    for line in text:
        wordLines.append(re.sub("[0-9]"," ", line))
    return wordLines
bowDb.CleanHeadline = rem_numbers(bowDb.Headline)
display(bowDb.head(5))

,Day,Month,Year,Source,Headline
0,1,'fevereiro',2017,'Valor','BNDES encolhe e volta ao nível de 20 anos atrás'
1,1,'fevereiro',2017,'Valor','BC cria novo instrumento de política monetária.'
2,1,'fevereiro',2017,'Valor','Câmbio gera bate-boca entre UA e UE.'
3,1,'fevereiro',2017,'Valor','Indenização a transmissoras de energia já che...
4,1,'fevereiro',2017,'Valor',"'Políticos esperam que relator separe ""joio do..."


Em sequência, podemos remover algumas palavras que não são relevantes para a análise, as _stopwords_. Para tanto, precisamos carregar esse conjunto disponível na biblioteca __nltk__:

In [10]:
from nltk.corpus import stopwords
print (stopwords.words("portuguese"))

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos

Com as _stopwords_ carregadas, podemos percorrer as manchetes e manter apenas as palavras que não pertencerem a esse grupo indesejado:

In [11]:
wordLines = []

for line in bowDb.Headline:
    words = ''
    for word in line.split():
        if word not in stopwords.words("portuguese"):
            words = words + word + ' '
    wordLines.append(words)
    
bowDb['CleanHeadline'] = wordLines
display(bowDb.head(5))

,Day,Month,Year,Source,Headline,CleanHeadline
0,1,'fevereiro',2017,'Valor','BNDES encolhe e volta ao nível de 20 anos atrás','BNDES encolhe volta nível 20 anos atrás'
1,1,'fevereiro',2017,'Valor','BC cria novo instrumento de política monetária.','BC cria novo instrumento política monetária.'
2,1,'fevereiro',2017,'Valor','Câmbio gera bate-boca entre UA e UE.','Câmbio gera bate-boca UA UE.'
3,1,'fevereiro',2017,'Valor','Indenização a transmissoras de energia já che...,'Indenização transmissoras energia chega tarif...
4,1,'fevereiro',2017,'Valor',"'Políticos esperam que relator separe ""joio do...","'Políticos esperam relator separe ""joio trigo"".'"


As frases ficam sem elementos de união, mas ainda transmitem seu completo sentido, como visto na primeira manchete, "BNDES encolhe volta nível 20 anos atrás".
<br>Por fim, podemos analisar as palavras de todas as manchetes e obter a polaridade para cada uma das palavras presentes no léxico. Para as frases em que nenhuma palavra estiver contida no léxico, o número 0 é atribuído, enquanto que para as demais, um vetor contendo as pontuações é considerado.

In [12]:
wordScores = []
lineScores = []
lineScore = 0
for line in bowDb.CleanHeadline:
    words = ''
    wordScores = []
    for word in line.split():
        correspWord = sentiLex[sentiLex.Word == word]
        if not correspWord.empty:
            wordScores.append(correspWord.Pol)
            
    if len(wordScores) != 0:
        lineScores.append(wordScores)
    else:
        lineScores.append(0)
bowDb['Scores'] = lineScores
display(bowDb.head(10))

,Day,Month,Year,Source,Headline,CleanHeadline,Scores
0,1,'fevereiro',2017,'Valor','BNDES encolhe e volta ao nível de 20 anos atrás','BNDES encolhe volta nível 20 anos atrás',0
1,1,'fevereiro',2017,'Valor','BC cria novo instrumento de política monetária.','BC cria novo instrumento política monetária.',[[0]]
2,1,'fevereiro',2017,'Valor','Câmbio gera bate-boca entre UA e UE.','Câmbio gera bate-boca UA UE.',0
3,1,'fevereiro',2017,'Valor','Indenização a transmissoras de energia já che...,'Indenização transmissoras energia chega tarif...,0
4,1,'fevereiro',2017,'Valor',"'Políticos esperam que relator separe ""joio do...","'Políticos esperam relator separe ""joio trigo"".'",0
5,1,'fevereiro',2017,'Valor','Philips quer administrar hospitais públicos n...,'Philips quer administrar hospitais públicos B...,0
6,1,'fevereiro',2017,'Valor','Com vendas em queda C&amp;C muda lojas e troc...,'Com vendas queda C&amp;C muda lojas troca dir...,0
7,1,'fevereiro',2017,'Globo','Fachin poderá ir para turma que julga Lava-Ja...,'Fachin poderá ir turma julga Lava-Jato.',0
8,1,'fevereiro',2017,'Globo','Eike tem multas que superam fundo para prisões.','Eike multas superam fundo prisões.',0
9,1,'fevereiro',2017,'Globo','Operador pagou decoração de luxo de imóveis d...,'Operador pagou decoração luxo imóveis Cabral.',0


No trecho avaliado acima, poucas frases apresentam resultados. A impressão que temos é que o vocabulário das manchetes é pouco representado no léxico. Para confirmar essa impressão, podemos montar o vocabulário das manchetes e comparar com o léxico. 
<br>Para essa operação, utilizamos o módulo _count vectorizer_ da biblioteca __scikit-learn__.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500000000) 
features = vectorizer.fit_transform(bowDb.CleanHeadline)
features = features.toarray()
print (features.shape)
vocabulary = vectorizer.get_feature_names()
print (vocabulary)

(500, 1633)
['abandonar', 'aberto', 'abertos', 'abertura', 'abilio', 'abismo', 'aborto', 'abre', 'abrigo', 'abrir', 'abuso', 'aceita', 'acelerar', 'acena', 'achaques', 'acidentes', 'acima', 'acordo', 'aderiu', 'adia', 'adiado', 'adiante', 'adiar', 'administrar', 'admite', 'admitiu', 'adriana', 'aeronáutica', 'aeroporto', 'afaga', 'afasta', 'afastar', 'afastou', 'afeta', 'afetar', 'afirma', 'agenda', 'agentes', 'agido', 'agora', 'agradar', 'agressão', 'agronegócio', 'ajudará', 'ajuste', 'al', 'ala', 'alas', 'alavanca', 'alcance', 'alckmista', 'alegorias', 'alemanha', 'aleppo', 'alerta', 'alguns', 'aliados', 'aloysio', 'alta', 'alternativa', 'alternativas', 'alto', 'altruísmo', 'aluguel', 'além', 'alíquota', 'amanhã', 'amarela', 'ameaça', 'ameaçou', 'amenizam', 'americana', 'americano', 'amil', 'amizade', 'amor', 'amp', 'amplia', 'américa', 'analistas', 'anatel', 'ancelmo', 'andrade', 'andreas', 'ano', 'anos', 'ansiedade', 'antes', 'anticorrupção', 'anuncia', 'anvisa', 'análises', 'anúnc

Obtemos um vocabulário com 1633 palavras distintas.
<br>Agora podemos ver quão representado no léxico esse vocabulário está.

In [14]:
# reference: from https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words
def count_word_occurences(features, vocabulary):
    dist = np.sum(features, axis=0)
    result = []
    for tag, count in zip(vocabulary, dist):
        result.append([count,tag])
    result.sort(reverse=True)
    return result

#print (count_word_occurences(features, vocabulary))

In [15]:
findings = 0
for word in vocabulary:
    correspWord = sentiLex[sentiLex.Word == word]
    if not correspWord.empty:
        findings += 1
print ("% de palavras do vocabulário no corpus:{}%".format((findings / len(vocabulary))*100))

% de palavras do vocabulário no corpus:6.307409675443969%


Como temos apenas 6.3% do vocabulário está contido no corpus sentiLex, é inviável utilizar o mesmo em nossa análise. Para contornar esse problema, poderíamos experimentar buscar no corpus _MacMorphos_ sinônimos/hiperônimos das palavras desconhecidas.

Ainda poderíamos considerar outro corpus em português, porém devido à escassez dos mesmos, optamos por traduzir as manchetes pra o inglês e buscar os resultados no corpus _SentiWordNet_, que é muito mais extenso.

Vale ressaltar que o corpus _SentiWordNet_ apresenta um aspecto interessante por não ter sido criado a partir de um único tipo de mídia, como avaliações de filmes ou produtos, mas sim a partir da avaliação sistemática de um outro corpus: o _wordnet_.

Para tanto, iniciamos removendo os numerais das manchetes com o intuito de eliminar elementos irrelevantes.

A documentação do _SentiWordNet_ está em: http://nmis.isti.cnr.it/sebastiani/Publications/LREC06.pdf

In [16]:
wordLines = []
for line in translatedDb.Headline:
    wordLines.append(re.sub("[0-9]"," ", line))
    
translatedDb.CleanHeadline = wordLines

Em sequência, para confirmarmos que o vocabulário em inglês é melhor representado no _SentiWordNet_, podemos montá-lo novamente e fazer a análise.

In [17]:
def get_vocabulary(text):
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 500000000) 
#Huge max_features, we don't want to limit ourserves

    features = vectorizer.fit_transform(text)
    features = features.toarray()
    #print (features.shape)
    voc = vectorizer.get_feature_names()
    return voc, features
enVoc = get_vocabulary(translatedDb.CleanHeadline)[0]

#print (enVoc)

from nltk.corpus import sentiwordnet as swn
count = 0
for word in enVoc:
    sentiList = list(swn.senti_synsets(word))
    if (len(sentiList) > 0):
        count += 1
print ("% de palavras do vocabulário no SentiWordNet:{0:.2f}%".format((count / len(enVoc))*100))

% de palavras do vocabulário no SentiWordNet:83.71%


Agora obtemos um resultado muito superior: 83.71% das palavras utilizadas estão contidas no corpus.
<br>Antes de classificar as palavras, podemos remover novamente as _stopwords_, desta vez considerando o conjunto para o inglês.

In [18]:
wordLines = []

for line in translatedDb.CleanHeadline:
    words = ''
    for word in line.split():
        if word not in stopwords.words("english"):
            words = words + word + ' '
    wordLines.append(words)

translatedDb['CleanEnHeadline'] = wordLines

Com essas informações, podemos obter a pontuação das palavras encontradas no corpus e calcular a média para cada sentença. 
<br>Uma observação relevante é que motivo de usarmos um _threshold_, uma margem de ativação em termos, é para criar um limiar mais brando de resultados medianos, e impedir que frases classificadas com leves valências positivas ou negativas estejam juntas com frases de valências fortes.

In [19]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)
wordScores = []
posWordScores = []
negWordScores = []
lineScores = []
lineScore = 0
THRESHOLD = 0.005
for line in translatedDb.CleanHeadline:
    words = ''
    wordScores = []
    for word in line.split():
        sentiList = list(swn.senti_synsets(word))
        if len(sentiList) > 0:
            
            posWordScores.append(sentiList[0].pos_score())
            negWordScores.append(sentiList[0].neg_score())
            
    if len(posWordScores) != 0: 
        posMean = mean(posWordScores)
        negMean = mean(negWordScores)
        if posMean > (negMean + THRESHOLD):
            result = 'pos'
        elif negMean > (posMean + THRESHOLD):
            result = 'neg'
        else:
            result = 'mid'
        lineScores.append([posMean, negMean, result ])
    else:
        lineScores.append([0,0])
                          
translatedDb['SW_Score'] = lineScores
display(translatedDb.head(5))

,Unnamed: 0.1,Day,Month,Year,Source,Headline,BlobSentimentsP,BlobSentimentsNB,CleanEnHeadline,SW_Score
0,0,1,'fevereiro',2017,'Valor','BNDES shrinks back to the level of 20 years ago',"Sentiment(polarity=0.0, subjectivity=0.0)","Sentiment(classification='pos', p_pos=0.879188...",'BNDES shrinks back level years ago',"[0.0, 0.0625, neg]"
1,1,1,'fevereiro',2017,'Valor','BC creates new monetary policy instrument.',"Sentiment(polarity=0.13636363636363635, subjec...","Sentiment(classification='pos', p_pos=0.989948...",'BC creates new monetary policy instrument.',"[0.046875, 0.03125, pos]"
2,2,1,'fevereiro',2017,'Valor','Exchange generates word of mouth between AU a...,"Sentiment(polarity=0.0, subjectivity=0.0)","Sentiment(classification='pos', p_pos=0.650384...",'Exchange generates word mouth AU EU.',"[0.028846153846153848, 0.028846153846153848, mid]"
3,3,1,'fevereiro',2017,'Valor','Indemnification to energy transmitters has al...,"Sentiment(polarity=0.0, subjectivity=0.0)","Sentiment(classification='pos', p_pos=0.575931...",'Indemnification energy transmitters already r...,"[0.027777777777777776, 0.020833333333333332, pos]"
4,4,1,'fevereiro',2017,'Valor',"""Politicians expect rapporteur to separate"" wh...","Sentiment(polarity=0.0, subjectivity=0.0)","Sentiment(classification='pos', p_pos=0.933297...","""Politicians expect rapporteur separate"" wheat...","[0.03409090909090909, 0.028409090909090908, pos]"


Com isso temos a classificação para cada uma das sentenças. Em um primeiro momento, é perceptível uma certa discrepância entre esse método e os outros dois apresentados anteriormente, como na manchete "BNDES encolhe e volta ao nível de 20 anos atrás", já que a última abordagem classificou a notícia como negativa, a pattern como neutra e a Naive Bayes como positiva.
<br>Este comportamento pode estar relacionado com a forma com que o banco de palavras do SentiWordNet foi montado.

# Vader

O analisador de texto Vader tem como base um vocabulário tipicamente usado em conversas informais, e é limitado à análise de frases únicas, sentenças isoladas. O banco de treinamento do Vader é treinado para compreender intensificadores de sentimentos (boosters), e utiliza bancos existentes em conjunto com muitas novas palavras e avaliações realizadas por humanos.
<br>Ele foi inicialmente imaginado como um analisador de mensagens de texto (sms) e frases em redes sociais, como o Twitter. A criação minuciosa de um novo banco de dados contendo gírias (slang) e emoticons permite uma análise mais correta desse tipo de mídia escrita informal, e consequentemente cria um bom ambiente de testes para manchetes curtas de jornal.

A documentação toda do analisador de sentimentos Vader está disponível em: https://github.com/cjhutto/vaderSentiment

In [20]:
vaderSent = []
from nltk.sentiment.vader import SentimentIntensityAnalyzer
for line in translatedDb.CleanEnHeadline:
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(line)
    vaderSent.append(ss)
translatedDb['Vader_Score'] = vaderSent
display(translatedDb.head(5))

,Unnamed: 0.1,Day,Month,Year,Source,Headline,BlobSentimentsP,BlobSentimentsNB,CleanEnHeadline,SW_Score,Vader_Score
0,0,1,'fevereiro',2017,'Valor','BNDES shrinks back to the level of 20 years ago',"Sentiment(polarity=0.0, subjectivity=0.0)","Sentiment(classification='pos', p_pos=0.879188...",'BNDES shrinks back level years ago',"[0.0, 0.0625, neg]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,1,1,'fevereiro',2017,'Valor','BC creates new monetary policy instrument.',"Sentiment(polarity=0.13636363636363635, subjec...","Sentiment(classification='pos', p_pos=0.989948...",'BC creates new monetary policy instrument.',"[0.046875, 0.03125, pos]","{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp..."
2,2,1,'fevereiro',2017,'Valor','Exchange generates word of mouth between AU a...,"Sentiment(polarity=0.0, subjectivity=0.0)","Sentiment(classification='pos', p_pos=0.650384...",'Exchange generates word mouth AU EU.',"[0.028846153846153848, 0.028846153846153848, mid]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,3,1,'fevereiro',2017,'Valor','Indemnification to energy transmitters has al...,"Sentiment(polarity=0.0, subjectivity=0.0)","Sentiment(classification='pos', p_pos=0.575931...",'Indemnification energy transmitters already r...,"[0.027777777777777776, 0.020833333333333332, pos]","{'neg': 0.0, 'neu': 0.533, 'pos': 0.467, 'comp..."
4,4,1,'fevereiro',2017,'Valor',"""Politicians expect rapporteur to separate"" wh...","Sentiment(polarity=0.0, subjectivity=0.0)","Sentiment(classification='pos', p_pos=0.933297...","""Politicians expect rapporteur separate"" wheat...","[0.03409090909090909, 0.028409090909090908, pos]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


Em uma análise preliminar, o Vader demonstra resultados superiores às outras implementações, embora uma análise detalhada é feita na seção de resultados.

In [21]:
translatedDb.to_csv('translatedDbScored.csv')

# Resultados
A fim de realizar uma análise comparativa de eficácia, selecionamos algumas manchetes e como esperávamos que elas iriam se comportar:
## Manchetes negativas

Escolhemos cinco manchetes com conotações negativas, para as quais esperávamos obter uma valência correspondentemente negativa, e analisamos os resultados de acordo com o esperado de cada uma.

In [22]:
n=43
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Elio Gaspari: Presidente bate recorde em perda de colaboradores.'
'Elio Gaspari: President sets record in loss of employees.'

TextBlob Pattern: 
Sentiment(polarity=0.0, subjectivity=0.0)

TextBlob Naive Bayes: 
Sentiment(classification='pos', p_pos=0.7905550485897339, p_neg=0.2094449514102658)

SentiWord: 
[0.04121621621621622, 0.04054054054054054, 'mid']

Vader: 
{'neg': 0.277, 'neu': 0.723, 'pos': 0.0, 'compound': -0.3182}


A matéria é claramente negativa, especialmente ao analisar o bigrama "record loss", ou o trigrama "record loss employees", mas "record" sozinha pode ser avaliada como positiva incorretamente.

Vemos que o TextBlob Pattern não conseguiu analisar nenhuma das palavras chave da manchete, provavelmente não contendo nenhuma das palavras em seu corpus. O TextBlob Naive Bayes avaliou a matéria como positiva, provavelmente ao analisar a palavra "record", que possui valência positiva maior do que a valência negativa de "loss". O SentiWord classificou a matéria como 'mid'. Já o Vader conseguiu pegar a nuância negativa, e apesar de ter um valor neutro alto, conseguiu deixar o compound em negativo.
Nesta manchete, percebemos que a utilização de valências palavra-a-palavra (unigramas) compromete certas construções de frases, especialmente quando há a presença de palavras intensificadoras que têm certa valência quando analisadas individualmente.
***

In [23]:
n=82
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Atentado mata 90 pessoas em Cabul.'
'Attack kills 90 people in Kabul.'

TextBlob Pattern: 
Sentiment(polarity=0.0, subjectivity=0.0)

TextBlob Naive Bayes: 
Sentiment(classification='neg', p_pos=0.4344497687542783, p_neg=0.5655502312457213)

SentiWord: 
[0.03596866096866097, 0.032407407407407406, 'mid']

Vader: 
{'neg': 0.767, 'neu': 0.233, 'pos': 0.0, 'compound': -0.765}


Manchete indica morte de 90 pessoas em Kabul, e a palavra "attack" pode ser mal-interpretada usando o banco de dados de análises de filmes.
Esta manchete também não é analisada corretamente pelo TextBlob Pattern, que a indica como puramente neutra. Já o TextBlob Naive Bayes classificou a manchete como negativa, embora tenha sido por uma margem pequena. O SentiWord analisou a matéria como positiva. Cremos que essas tendências ao positivo inclua a palavra "ataque", que sem contexto não pode ser atribuída a terrorismos ou atentados, e o lado negativo seja graças à palavra "kills". O Vader conseguiu avaliar a matéria como altamente negativa, dando um score composto bastante negativo. De um ponto de vista básico, o Vader foi o mais preciso, ao atribuir um score negativo alto e um composto devidamente negativo.
***

In [24]:
n=125
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Homem mata a tiros ex-mulher filho e mais dez.'
'Man kills the ex-wife son and ten more.'

TextBlob Pattern: 
Sentiment(polarity=0.25, subjectivity=0.25)

TextBlob Naive Bayes: 
Sentiment(classification='neg', p_pos=0.3817142012032676, p_neg=0.6182857987967325)

SentiWord: 
[0.03704588910133843, 0.04206500956022945, 'neg']

Vader: 
{'neg': 0.412, 'neu': 0.588, 'pos': 0.0, 'compound': -0.5423}


O uso da palavra "kills" deveria indicar a valência negativa, mas o contexto de nomear "son", "ex-wife", "more" pode confundir o processamento.
Nesta manchete, ambos o TextBlob Naive Bayes e o SentiWord conseguiram classificar a matéria como negativa, enquanto o TextBlob Pattern colocou a valência em 0.25, o que corresponderia a uma valência positiva. Possivelmente o uso da palavra "more" puxou a valência para o positivo, e "kills" para o negativo. A análise do Vader conseguiu mais uma vez classificar corretamente como negativo, com um score final de -0.5, que indicaria uma manchete bastante negativa.
***

In [25]:
n=406
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Ataque suicida matou 80 crianças.'
'Suicide attack killed 80 children.'

TextBlob Pattern: 
Sentiment(polarity=-0.2, subjectivity=0.0)

TextBlob Naive Bayes: 
Sentiment(classification='neg', p_pos=0.4280023076587344, p_neg=0.5719976923412652)

SentiWord: 
[0.038643533123028394, 0.04305993690851735, 'mid']

Vader: 
{'neg': 0.924, 'neu': 0.076, 'pos': 0.0, 'compound': -0.9201}


Manchete negativa especialmente pelo uso das palavras "suicide", "attack", "killed". "Children" pode confundir o processamento, bem como "attack" pode ser mal-interpretado sem o contexto apropriado.
A manchete que possuía a palavra-chave "suicídio" foi classificada negativa pelo TextBlob Pattern e pelo TextBlob Naive Bayes, mas teve classificação "mid" pelo SentiWord, possivelmente tendo sido combatida pela valência positiva de "crianças" avaliada independentemente. Ter usado algo maior do que unigramas aqui teria feito diferença em identificar o contexto das crianças. Pelo Vader, temos um dos maiores scores negativos, com um score final de -0.92, que está próximo do mais negativo possível (-1.00).
***

In [26]:
n=485
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Ataque a mesquita no Canadá deixa ao menos seis mortos.'
'Attack on mosque in Canada leaves at least six dead.'

TextBlob Pattern: 
Sentiment(polarity=-0.25, subjectivity=0.4)

TextBlob Naive Bayes: 
Sentiment(classification='neg', p_pos=0.3698658755476561, p_neg=0.6301341244523436)

SentiWord: 
[0.03976272066458982, 0.04227258566978193, 'mid']

Vader: 
{'neg': 0.341, 'neu': 0.659, 'pos': 0.0, 'compound': -0.4767}


Palavras-chave são "attack", que pode acabar tendo valências diferentes dependendo do contexto, e "kills", que é geralmente negativa.
Caso similar ao anterior, TextBlob Pattern e TextBlob Naive Bayes classificaram corretamente a manchete como negativa, bem como o Vader, mas o SentiWord ficou em "mid", com um score negativo levemente mais alto que o positivo. 
***

## Manchetes positivas

Notamos dificuldade em encontrar manchetes com conotação positiva, então optamos por selecionar aquelas que tinham palavras específicas que poderiam ser consideradas positivas, tentando prever o comportamento do algoritmo.

In [27]:
n=32
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Doria atribui doações a altruísmo empresarial.'
'Doria attributes donations to corporate altruism.'

TextBlob Pattern: 
Sentiment(polarity=0.0, subjectivity=0.0)

TextBlob Naive Bayes: 
Sentiment(classification='neg', p_pos=0.3181818181818187, p_neg=0.681818181818181)

SentiWord: 
[0.037162162162162164, 0.03885135135135135, 'mid']

Vader: 
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


A manchete deve ser avaliada como positiva dadas as palavras-chave "altruism", "donations", mas vemos que alguns algoritmos não conseguem identificar essas nuâncias. O TextBlob Naive Bayes classificou incorretamente como negativa, o SentiWord como mid, e o Vader e o TextBlob Pattern não conseguiram sair de um resultado de total neutralidade.
***

In [28]:
n=242
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Com FGTS varejo volta a crescer.'
'With FGTS retail grow back.'

TextBlob Pattern: 
Sentiment(polarity=0.0, subjectivity=0.0)

TextBlob Naive Bayes: 
Sentiment(classification='neg', p_pos=0.49989808397880164, p_neg=0.5001019160211981)

SentiWord: 
[0.037435233160621764, 0.04015544041450777, 'mid']

Vader: 
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


A manchete em português era positiva, anunciando o crescimento do varejo graças ao FGTS, mas em inglês essa nuância é perdida, embora ainda contenha "grow back" como palavras-chave, que deveriam puxar para o positivo. Todos estão quase que perfeitamente no neutro, e a classificação do TextBlob Naive Bayes é graças a uma diferença de 0.0001 a mais no score negativo.
***

In [29]:
n=257
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Relator quer MP do Refis mais favorável a empresas.'
'Rapporteur wants MP of Refis more favorable to companies.'

TextBlob Pattern: 
Sentiment(polarity=0.35, subjectivity=0.3)

TextBlob Naive Bayes: 
Sentiment(classification='pos', p_pos=0.8157045314347129, p_neg=0.18429546856528692)

SentiWord: 
[0.0384521484375, 0.03955078125, 'mid']

Vader: 
{'neg': 0.0, 'neu': 0.617, 'pos': 0.383, 'compound': 0.4767}


Manchete que contém palavras "favorable", "wants", que deveria puxar para o positivo considerando as palavras independentemente.
Vemos que o TextBlob Pattern conseguiu identificar algumas palavras e classificar a manchete como positiva, bem como o TextBlob Naive Bayes, que colocou a manchete em um score bem alto positivo. A análise do Vader classificou a manchete como bastante positiva, e o SentiWord a colocou no mid.
De uma maneira geral, as palavras nesta manchete conseguiram ser identificadas mais facilmente pelos algoritmos.
***

In [30]:
n=283
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'XP quer ser líder em cinco anos.'
'XP wants to be a leader in five years.'

TextBlob Pattern: 
Sentiment(polarity=0.2, subjectivity=0.1)

TextBlob Naive Bayes: 
Sentiment(classification='pos', p_pos=0.6447161967612489, p_neg=0.3552838032387515)

SentiWord: 
[0.03872462488967343, 0.040158870255957636, 'mid']

Vader: 
{'neg': 0.0, 'neu': 0.545, 'pos': 0.455, 'compound': 0.516}


Algoritmo deve identificar as palavras "wants", "leader", embora a manchete não seja total e empiricamente boa. Vemos esse resultado em todos os métodos exceto o SentiWord, que classifica a manchete como levemente mais negativa.
***

In [31]:
n=332
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Casos de suspeita de dengue e zika caem em média 90% no ano.'
'Cases of suspected dengue fever and zika fall on average 90% in the year.'

TextBlob Pattern: 
Sentiment(polarity=-0.15, subjectivity=0.39999999999999997)

TextBlob Naive Bayes: 
Sentiment(classification='pos', p_pos=0.6333116741900876, p_neg=0.36668832580991295)

SentiWord: 
[0.038526752072343635, 0.04163526752072343, 'mid']

Vader: 
{'neg': 0.213, 'neu': 0.787, 'pos': 0.0, 'compound': -0.2263}


Manchete na qual o contexto das palavras leva a uma notícia empiricamente boa, mas há uma dificuldade dos algoritmos de identificar isso como uma coisa boa. Palavras-chave "fall", "fever", "suspected", que podem levar para o negativo. O método de Naive Bayes conseguiu prever adequadamente a classificação positiva, enquanto os outros todos classificaram como mais negativo do que positivo.
***

## Manchetes escolhidas aleatoriamente

Para estas cinco manchetes a seguir, utilizamos um algoritmo gerador de números aleatórios, eliminando quaisquer manchetes que já tivessem sido analisadas nos casos anteriores.

In [32]:
n=66
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Joesley deu dinheiro a firma da qual filho de Mantega foi sócio.'
'Joesley gave money to the firm of which Mantega's son was a partner.'

TextBlob Pattern: 
Sentiment(polarity=-0.2, subjectivity=0.4)

TextBlob Naive Bayes: 
Sentiment(classification='pos', p_pos=0.5122892109143866, p_neg=0.4877107890856142)

SentiWord: 
[0.03482142857142857, 0.03616071428571429, 'mid']

Vader: 
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


À primeira vista, não há nenhuma palavra-chave que funcione muito bem sem contexto aplicado. "Gave" pode ser um indicativo de positivo, bem como "partner". Vemos que o TextBlob Pattern analisa como negativo, mas com alto índice de subjetividade. O método de Naive Bayes analisa como positivo.
***

In [33]:
n=247
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Sérgio Cabral é condenado a 14 anos na Lava-Jato.'
'Sérgio Cabral is sentenced to 14 years in the Lava-Jet.'

TextBlob Pattern: 
Sentiment(polarity=0.0, subjectivity=0.0)

TextBlob Naive Bayes: 
Sentiment(classification='neg', p_pos=0.4861680150974352, p_neg=0.5138319849025648)

SentiWord: 
[0.03765368852459016, 0.04008709016393443, 'mid']

Vader: 
{'neg': 0.216, 'neu': 0.784, 'pos': 0.0, 'compound': -0.0258}


Palavra-chave "sentenced", e embora seja algo subjetivamente positivo, a palavra carrega uma conotação negativa quando avaliada independentemente.
Fora o TextBlob Pattern, todos os algoritmos consideraram a manchete como mais negativa do que positiva. O Vader identificou apenas palavras negativas e neutras.
***

In [34]:
n=329
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Ex-presidente depõe e diz que é vítima de "quase um massacre".'
'Former president testifies and says he is the victim of' almost a massacre '. "

TextBlob Pattern: 
Sentiment(polarity=-0.037500000000000006, subjectivity=0.025)

TextBlob Naive Bayes: 
Sentiment(classification='neg', p_pos=0.4330732470389665, p_neg=0.5669267529610326)

SentiWord: 
[0.039116296863045144, 0.041507268553940324, 'mid']

Vader: 
{'neg': 0.208, 'neu': 0.792, 'pos': 0.0, 'compound': -0.2732}


Palavra-chave forte "massacre", juntamente com "victim", o que deve levar a análise automática para uma análise que resulta em perceber uma conotação negativa aparente. Todos os algoritmos identificaram um sentimento negativo mais forte, graças às palavras acima.
***

In [35]:
n=347
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Velloso pode comandar a Justiça.'
'Velloso can command justice.'

TextBlob Pattern: 
Sentiment(polarity=0.0, subjectivity=0.0)

TextBlob Naive Bayes: 
Sentiment(classification='pos', p_pos=0.8247275974319552, p_neg=0.17527240256804413)

SentiWord: 
[0.038461538461538464, 0.041908563134978226, 'mid']

Vader: 
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


A manchete tem palavras positivas, como "command" e "justice", então é esperado que o algoritmo identifique a valência positiva da frase, mas vemos isso somente no TextBlob Naive Bayes. O TextBlob Pattern e o Vader não conseguiram sair do neutro, e o SentiWord identificou a manchete como mais negativa do que positiva.
***

In [36]:
n=483
print(originalDb.iloc[n].Headline)
print(translatedDb.iloc[n].Headline)
print('\nTextBlob Pattern: \n'+translatedDb.iloc[n].BlobSentimentsP)
print('\nTextBlob Naive Bayes: \n'+translatedDb.iloc[n].BlobSentimentsNB+'\n')
print('SentiWord: ')
print(translatedDb.SW_Score.iloc[n])
print('\nVader: ')
print(translatedDb.Vader_Score.iloc[n])

'Eike se entrega e indca que vai colaborar com investigações.'
'Eike surrenders and indicates that he will collaborate with investigations.'

TextBlob Pattern: 
Sentiment(polarity=0.0, subjectivity=0.0)

TextBlob Naive Bayes: 
Sentiment(classification='pos', p_pos=0.975638392557124, p_neg=0.02436160744287506)

SentiWord: 
[0.03994670846394984, 0.042537617554858936, 'mid']

Vader: 
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


Palavras-chave positivas devem pesar mais do que as negativas, no caso "surrenders" carrega uma valência provavelmente positiva, bem como "collaborate", verbos de ação que indicam uma ação positiva. Vemos que o TextBlob Pattern e o Vader não saíram do neutro mais uma vez, mas o TextBlob Naive Bayes deu o maior score positivo já visto nos nossos resultados.
***

# Pontos Fracos e Fortes
Após analisar cada uma das quinze manchetes escolhidas, podemos obter algumas conclusões a respeito dos quatro algoritmos que usamos.

Vimos que a **TextBlob Pattern** é, em termos, a mais rudimentar delas. Capaz de avaliar a polaridade de -1.0 (mais negativa possível) a 1.0 (mais positiva possível), mas incapaz de determinar contextos como vimos em muitos dos resultados.
<br>Cremos que isso possa acontecer devido ao fato do vocabulário de avaliações de produtos terem, normalmente, um léxico menor do que outros bancos de dados.

O **TextBlob Naive Bayes** possui um resultado coerente na maioria das vezes, mas possui alguns "falsos positivos" ao utilizar um corpus de avaliações de filmes. Teorizamos que algumas das palavras usadas negativamente em manchetes podem ser usadas em avaliações boas de filmes, tal como já mencionado que "medo" e "susto" podem ser palavras usadas para ilustrar aspectos positivos de um filme de terror.
<br>Podemos perceber esta influência do conjunto de treinamento de avaliações de filmes em manchetes como: 

_'Fear of more repression empties streets of Asunción.'_
Sentiment(classification='**pos**', p_pos=0.904, p_neg=0.096)

_'Young people study in fear.'_
Sentiment(classification='**pos**', p_pos=0.82, p_neg=0.18)


O **SentiWordNet** é um banco de palavras criado a partir de uma atribuição manual, e usa unigramas para a análise. Podemos ver esse resultado em várias das manchetes, nas quais algumas poucas palavras determinam incorretamente o resultado do sentimento da frase.

Para frases curtas, o **Vader** apresenta resultados coerentes graças às suas atribuições de palavras intensificadoras e sua capacidade de analisar o contexto da frase toda. Um ponto fraco é seu pequeno dicionário de valências. Se ele fosse atualizado com mais algumas centenas de palavras, poderia melhorar bastante suas já ótimas análises.


# Lições Aprendidas

Pudemos perceber que analisar o contexto das frases é essencial, e a utilização de unigramas compromete de certa forma a análise como um todo. O uso de palavras intensificadoras (_boosters_) contribui bastante para um resultado coerente, embora ainda não estejamos no nível de captar certas nuâncias implícitas pelo contexto.


# Trabalhos Futuros


Futuramente, é possível implementar funções que determinem a valência de uma sentença a partir da avaliação de diversões classificadores distintos, unificando o resultado.
<br>Vemos que, com uma base desta, é possível criar uma série de sistemas que usam os resultados de análises de sentimentos para comparar canais de comunicação em diversas vertentes. Podemos por exemplo avaliar se um site de notícias é muito negativo, ou ver se um dia em especial teve uma valência muito positiva para todos os jornais, dentre outras possibilidades.
<br> Por fim, existe a possibilidade de futuramente implementar aspectos de automatização para este trabalho, como processar bancos de dados originais diretamente de arquivos CSV, tratar erros de digitação nestes bancos, e até mesmo obter novas manchetes diretamente de sites de notícias.

# Bônus

## Análises dos Jornais

### Análise dos vocabulários

Nosso primeiro passo é isolar as manchetes de cada jornal:

In [37]:
valor = originalDb[originalDb.Source=='\'Valor\'']
folha = originalDb[originalDb.Source=='\'Folha\'']
estado = originalDb[originalDb.Source == '\'Estado\'']
globo = originalDb[originalDb.Source == '\'Globo\'']

print ('Temos: \n {0} Manchetes da Folha \n {1} Manchetes do Valor \n {2} Manchetes do Estado\n {3} Manchetes do Globo'
       .format(len(folha), len(valor), len(estado), len(globo)))

Temos: 
 127 Manchetes da Folha 
 118 Manchetes do Valor 
 129 Manchetes do Estado
 126 Manchetes do Globo


Com as manchetes isoladas, é possível montarmos o vocabulário específico de cada jornal.

In [38]:
valorVoc, valorFeat = get_vocabulary(valor.Headline)
folhaVoc, folhaFeat = get_vocabulary(folha.Headline)
estadoVoc, estadoFeat = get_vocabulary(estado.Headline)
globoVoc, globoFeat = get_vocabulary(globo.Headline)
def show_most_used_words(vocabulary, features):
    return (count_word_occurences(features, vocabulary)[:20])

print("Mais usadas no Valor: \n {}".format(show_most_used_words(valorVoc, valorFeat)))
print("Mais usadas na Folha: \n {}".format(show_most_used_words(folhaVoc, folhaFeat)))
print("Mais usadas no Estado: \n {}".format(show_most_used_words(estadoVoc, estadoFeat)))
print("Mais usadas no Globo: \n {}".format(show_most_used_words(globoVoc, globoFeat)))

Mais usadas no Valor: 
 [[40, 'de'], [20, 'da'], [16, 'para'], [16, 'do'], [9, 'no'], [7, 'novo'], [7, 'governo'], [7, 'em'], [6, 'temer'], [6, 'mais'], [5, 'trump'], [5, 'já'], [5, 'brasil'], [5, 'ao'], [4, 'vão'], [4, 'volta'], [4, 'se'], [4, 'que'], [4, 'por'], [4, 'pode']]
Mais usadas na Folha: 
 [[57, 'de'], [26, 'em'], [26, 'da'], [16, 'do'], [14, 'no'], [13, 'que'], [13, 'para'], [13, 'na'], [9, 'temer'], [9, 'diz'], [8, 'não'], [7, 'mais'], [7, 'com'], [6, 'um'], [6, 'sem'], [6, 'se'], [6, 'presidente'], [6, 'governo'], [6, 'doria'], [6, 'deve']]
Mais usadas no Estado: 
 [[51, 'de'], [21, 'do'], [18, 'em'], [14, 'da'], [11, 'na'], [10, 'para'], [10, 'no'], [9, 'com'], [8, 'trump'], [7, 'temer'], [7, 'que'], [7, 'diz'], [6, 'quer'], [5, 'sobre'], [5, 'mantém'], [5, 'dos'], [4, 'vai'], [4, 'stf'], [4, 'sp'], [4, 'por']]
Mais usadas no Globo: 
 [[42, 'de'], [19, 'da'], [16, 'do'], [12, 'na'], [12, 'em'], [10, 'para'], [10, 'no'], [9, 'temer'], [7, 'vai'], [6, 'trump'], [6, 'com'],

Como podemos ver acima, as palavras mais recorrentes são em sua maioria _stopwords_.
Para uma análise melhor, podemos removê-las e reavaliar os resultados.

In [39]:
bowDb.CleanHeadline
valor = bowDb[bowDb.Source=='\'Valor\'']
valorVoc, valorFeat = get_vocabulary(valor.CleanHeadline)
folha = bowDb[bowDb.Source=='\'Folha\'']
folhaVoc, folhaFeat = get_vocabulary(folha.CleanHeadline)
estado = bowDb[bowDb.Source == '\'Estado\'']
estadoVoc, estadoFeat = get_vocabulary(estado.CleanHeadline)
globo = bowDb[bowDb.Source == '\'Globo\'']
globoVoc, globoFeat = get_vocabulary(globo.CleanHeadline)
print("\nMais usadas no Valor: \n {}".format(show_most_used_words(valorVoc, valorFeat)))
print("\nMais usadas na Folha: \n {}".format(show_most_used_words(folhaVoc, folhaFeat)))
print("\nMais usadas no Estado: \n {}".format(show_most_used_words(estadoVoc, estadoFeat)))
print("\nMais usadas no Globo: \n {}".format(show_most_used_words(globoVoc, globoFeat)))


Mais usadas no Valor: 
 [[7, 'novo'], [7, 'governo'], [6, 'temer'], [5, 'trump'], [5, 'brasil'], [4, 'vão'], [4, 'volta'], [4, 'pode'], [4, 'eua'], [4, 'diz'], [4, 'bi'], [3, 'vê'], [3, 'us'], [3, 'ser'], [3, 'quer'], [3, 'política'], [3, 'país'], [3, 'odebrecht'], [3, 'mercado'], [3, 'indenização']]

Mais usadas na Folha: 
 [[9, 'temer'], [9, 'diz'], [6, 'presidente'], [6, 'governo'], [6, 'doria'], [6, 'deve'], [5, 'trump'], [5, 'ter'], [5, 'país'], [5, 'menos'], [5, 'crise'], [5, 'contra'], [5, 'brasil'], [4, 'stf'], [4, 'sp'], [4, 'sobre'], [4, 'odebrecht'], [4, 'nova'], [4, 'maduro'], [4, 'lula']]

Mais usadas no Estado: 
 [[8, 'trump'], [7, 'temer'], [7, 'diz'], [6, 'quer'], [5, 'sobre'], [5, 'mantém'], [4, 'vai'], [4, 'stf'], [4, 'sp'], [4, 'pede'], [4, 'governo'], [4, 'eua'], [4, 'brasil'], [4, 'anos'], [3, 'volta'], [3, 'vezes'], [3, 'total'], [3, 'supremo'], [3, 'supera'], [3, 'reforma']]

Mais usadas no Globo: 
 [[9, 'temer'], [7, 'vai'], [6, 'trump'], [4, 'rio'], [4, 'pode'

Agora podemos ver com mais clareza o conjunto de palavras mais utilizadas por cada jornal. Percebemos algumas características interessantes como:
- Região principal do jornal: percebemos que a _Folha de São Paulo_ possui um forte foco na cidade de São Paulo, perceptível pelo grande número de menções ao prefeito João Doria. Já o jornal _O Globo_ possui 4 menções ao prefeito da cidade do Rio de Janeiro, Marcelo Crivella;
- Trump: todos os jornais possuem menções do novo presidente dos EUA, Donald Trump, em alta recorrência. O _Estado de São Paulo_ tem inclusive maior recorrência de mencionar o Trump do que o atual presidente do Brasil, Michel Temer.
- Odebrecht: Com a exceção do jornal _O Estado de São Paulo_, todos os jornais mencionam a Odebrecht em alta recorrência, graças às relações da empresa com propinas de políticos, assunto que está em alta no cenário político-jornalístico do Brasil.

## Análise de valência mais recorrente

Nessa seção, analisamos a valências médias por jornal. Para tanto, utilizaremos os resultados obtidos com a abordagem _Vader_, realizando a devida conversão para um formato mais amigável para a tarefa.

In [40]:
vaderS = translatedDb.Vader_Score
#A lambda function would be much better
neg = []
pos = []
neu = []
for score in vaderS:
    neg.append(score['neg'])
    pos.append(score['pos'])
    neu.append(score['neu'])
translatedDb['VaderNeg'] = neg
translatedDb['VaderPos'] = pos
translatedDb['VaderNeu'] = neu
valor = translatedDb[translatedDb.Source=='\'Valor\'']
folha = translatedDb[translatedDb.Source=='\'Folha\'']
estado = translatedDb[translatedDb.Source == '\'Estado\'']
globo = translatedDb[translatedDb.Source == '\'Globo\'']
def get_means(db):
    print("Neg: {0}, Pos: {1}, Neu:{2}".format(db.VaderNeg.mean(), db.VaderPos.mean(), db.VaderNeu.mean()))
print ('Globo Means: ')
get_means(globo)
print ('Valor Means: ')
get_means(valor)
print ('Folha Means: ')
get_means(folha)
print ('Estado Means: ')
get_means(estado)

Globo Means: 
Neg: 0.13594444444444448, Pos: 0.0982063492063492, Neu:0.7658571428571429
Valor Means: 
Neg: 0.09355084745762711, Pos: 0.08860169491525424, Neu:0.8178474576271186
Folha Means: 
Neg: 0.19123622047244093, Pos: 0.09375590551181102, Neu:0.7150078740157482
Estado Means: 
Neg: 0.13589922480620154, Pos: 0.1213798449612403, Neu:0.7427364341085271


Os resultados demonstram que o Valor é o jornal com manchetes com valência mais neutras e menos de positivas ou negativas. Por outro lado, a Folha de São Paulo é o jornal com o menor média de pontuação neutra e maior média de pontuação negativa. Já o jornal mais otimista é o Estado de São Paulo, mesmo considerando que a média de valências negativas é maior do que a de positivas.

## Descobridor de jornal

In [41]:
valor = originalDb[originalDb.Source=='\'Valor\'']
folha = originalDb[originalDb.Source=='\'Folha\'']
estado = originalDb[originalDb.Source == '\'Estado\'']
globo = originalDb[originalDb.Source == '\'Globo\'']

print ('Temos: \n {0} Manchetes da Folha \n {1} Manchetes do Valor \n {2} Manchetes do Estado\n {3} Manchetes do Globo'
       .format(len(folha), len(valor), len(estado), len(globo)))

Temos: 
 127 Manchetes da Folha 
 118 Manchetes do Valor 
 129 Manchetes do Estado
 126 Manchetes do Globo


Como o número de manchetes de cada jornal está equilibrado, não precisamos tomar nenhum cuidado adicional nesse sentido. 

Vamos separar em conjuntos de treinamento e de teste, com 40% dos dados para teste e 60% para treinamento:

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(originalDb.Headline, originalDb.Source, test_size=0.4, random_state=7)


Agora podemos montar nossos modelos. Utilizaremos novamente o countVectorizer para a separação das palavras, porém dessa vez também utilizaremos o algoritmo Tf-idf para obter a relevância de uma palavra em relação à coleção de palavras do documento e uma implementação de Naive Bayes multinomial. 

In [43]:
#http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tf, y_train)

Para facilitar o processo de adequação/treinamento, criaremos um pipeline com os três algoritmos.

In [44]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])
text_clf.fit(X_train, y_train)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [45]:
docs_test = X_test
predicted = text_clf.predict(docs_test)
np.mean(predicted == y_test) 

0.28499999999999998

Obtemos um classificador com baixa performance relativa, embora este resultado esteja dentro do esperado ao considerarmos o tamanho do dataset de treinamento e de teste. 
Podemos tentar otimizar nosso classificador, utilizando busca extensiva de parâmetros.

In [46]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1,3), (1,4), (1,5),
                                   (2,1), (2,2), (2,3), (2,4)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3, 0.1, 0.0001, 0.5, 1),
}
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)
gs_clf.best_score_                                  
for param_name in sorted(parameters.keys()):
     print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.01
tfidf__use_idf: True
vect__ngram_range: (1, 4)


In [47]:
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,4))),
                      ('tfidf', TfidfTransformer(use_idf=True)),
                      ('clf', MultinomialNB(alpha=0.01)),
])
text_clf.fit(X_train, y_train) 
docs_test = X_test
predicted = text_clf.predict(docs_test)
np.mean(predicted == y_test) 

0.32000000000000001

Com a otimização, percebemos que o ideal é utilizarmos conjuntos de 4 palavras por análise, **alpha** para Naive Bayes multinomial igual à 0.01 e algoritmo **idf**, obtendo assim precisão de 32%.

In [48]:
def find_source(docs_test):
    predicted = text_clf.predict_proba(docs_test)
    print(docs_test)
    print(text_clf.classes_)
    print(predicted, '\n')
headlines_test = [["Amo o Dória"],
                ["Dólar caro"], 
                ["Fracasso Lava Jato"], 
                ["desemprego sobe"],
                ["violência sem limites"], 
                ["trump sucesso"],
                ["temer lixo no brasil"],
                ["T2 nota dez"]
                 ]
for headline in headlines_test:
    find_source(headline)

['Amo o Dória']
["'Estado'" "'Folha'" "'Globo'" "'Valor'"]
[[ 0.24333333  0.27333333  0.24666667  0.23666667]] 

['Dólar caro']
["'Estado'" "'Folha'" "'Globo'" "'Valor'"]
[[ 0.87303683  0.03833311  0.04572673  0.04290333]] 

['Fracasso Lava Jato']
["'Estado'" "'Folha'" "'Globo'" "'Valor'"]
[[ 0.44780194  0.36895537  0.1819543   0.00128839]] 

['desemprego sobe']
["'Estado'" "'Folha'" "'Globo'" "'Valor'"]
[[ 0.11324043  0.06723401  0.81046446  0.0090611 ]] 

['violência sem limites']
["'Estado'" "'Folha'" "'Globo'" "'Valor'"]
[[ 0.17501704  0.61424617  0.20066602  0.01007077]] 

['trump sucesso']
["'Estado'" "'Folha'" "'Globo'" "'Valor'"]
[[ 0.39591828  0.21353699  0.29996875  0.09057598]] 

['temer lixo no brasil']
["'Estado'" "'Folha'" "'Globo'" "'Valor'"]
[[ 0.22202333  0.02933653  0.00855506  0.74008508]] 

['T2 nota dez']
["'Estado'" "'Folha'" "'Globo'" "'Valor'"]
[[ 0.82502964  0.06816217  0.01094132  0.09586686]] 

